In [99]:

import json
from pandas import json_normalize

data_infile = 'open-data/data/events/'
match_id = 7298
data_path = data_infile+ str(match_id) + '.json'


with open(data_path) as f:
    data = json.load(f)

In [100]:



players = {}

for event in data: 
    if event.get('type', {}).get('name') == 'Starting XI': 
        lineup = event.get('tactics', {}).get('lineup', [])
        for entry in lineup: 
            player = entry['player']
            players[player['name']] = player['id']


In [101]:
def shot_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    outcome = event.get('interception', {}).get('outcome', '')
    return f"Shot py {player} of team{team} from {start_location}, {outcome}"
    
def offside_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    return f"{player} of {team} caught offsides"

def interception_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    outcome = event.get('interception', {}).get('outcome', '')
    return f"Interception {outcome} by {player} of {team} at {start_location}" 
    
def foul_won_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f'Foul won by {player} of team {team} at {start_location}'

def foul_committed_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f'{player} of team {team} committed foul at {start_location}'
def dribbled_past(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f'{player} of team {team} dribbled past at {start_location}'

def dribble_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    outcome = event.get('dribble', '').get('outcome', '')
    return f'{outcome} dribble by {player} of team {team} at {start_location}'

def dispossed_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    return f"{player} of team {team} dispossed at {start_location}"

def clearance_event(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    play_pattern = event.get('location', [])
    return f'Clearance by {player} of team {team} from {start_location} in {play_pattern}'

def block_event(event):
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    play_pattern = event.get('play_pattern').get('name', '')
    return f"Block by {player} of team {team} at {start_location} from a {play_pattern}"

def carry_event(event):
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])
    play_pattern = event.get('play_pattern').get('name', '')
    duration = event.get('duration', 0)
    end_location = event.get('carry', {}).get('end_location', [])
    return f"Carry by {player} of team {team} from {start_location} to {end_location} lasting {duration}"

def pass_event(event):
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    pass_details = event.get('pass', {})
    pass_type = pass_details.get('height', {}).get('name', 'Ground Pass')
    length = round(pass_details.get('length', 0), 1)
    start_location = event.get('location', [])
    end_location = pass_details.get('end_location', [])
    under_pressure = event.get('under_pressure', False)
    pressure_note = " under pressure" if under_pressure else ""

    recipient = pass_details.get('recipient', {}).get('name')
    outcome = pass_details.get('outcome', {}).get('name')  # e.g., 'Incomplete'

    if recipient and not outcome:
        return f"{player} of {team} makes a {length}m {pass_type.lower()} to {recipient} from {start_location} to {end_location}{pressure_note}."
    elif outcome:
        return f"{player} of {team} attempts a {length}m {pass_type.lower()} from {start_location}{pressure_note}, but it is {outcome.lower()}."
    else:
        return f"{player} of {team} plays a {length}m {pass_type.lower()} from {start_location} to {end_location}{pressure_note}."
    
def ball_recovery(event): 
    player = event.get('player', {}).get('name', 'Unknown Player')
    team = event.get('team', {}).get('name', '')
    start_location = event.get('location', [])

    return f"Ball Recovery by {player} of team {team} at {start_location}"


In [102]:
EVENT_DISPATCH = {
    'Shot': shot_event,
    'Offside': offside_event,
    'Interception': interception_event,
    'Foul Won': foul_won_event,
    'Foul Committed': foul_committed_event,
    'Dribbled Past': dribbled_past,
    'Dribble': dribble_event,
    'Dispossessed': dispossed_event,
    'Clearance': clearance_event,
    'Block': block_event,
    'Carry': carry_event, 
    'Pass': pass_event
}


In [103]:
def process_event(event):
    event_type = event.get('type', {}).get('name')
    formatter = EVENT_DISPATCH.get(event_type)
    if formatter:
        return formatter(event)
    return None  


In [104]:
prompts = []
for event in data:
    prompt = process_event(event)
    if prompt: 
        prompt = "I am going to describe a football event and I want you to generate a piece of commentary for that event: " + prompt
        prompts.append(prompt)
    
print(prompts[0])


I am going to describe a football event and I want you to generate a piece of commentary for that event: So-Yun Ji of Chelsea FCW makes a 3.6m ground pass to Ramona Bachmann from [61.0, 40.0] to [63.0, 37.0].


In [105]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",     # uses GPU if available
    torch_dtype="auto",    # use float16 if possible
    trust_remote_code=True
)

llama = pipeline("text-generation", model=model, tokenizer=tokenizer)


response = llama(prompts[0], max_new_tokens=100, do_sample=True, temperature=0.9)
print(response[0]['generated_text'])


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-6824eccc-1c52c47761ccc1143b23027b;d18dac5a-5ef1-4136-ab3e-3de0a9ca39bc)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.